In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from shapely.geometry import Point

%matplotlib inline

In [18]:
#the WHO data has quotes around all of the values and all values are objects. Removing the quotes around the fields that are
#dates or numbers to make it easier to use for calculations. The only date field I can convert to datetime without error is registration.

WHO_all = pd.read_csv('../data/WHO_from_python.csv') 
WHO_all['Date_registration'] = WHO_all['Date_registration'].str.replace('"','') 
WHO_all['Date_enrollement'] = WHO_all['Date_enrollement'].str.replace('"','') 
WHO_all['Target_size'] = WHO_all['Target_size'].str.replace('"','')
WHO_all['Date_registration'] = pd.to_datetime(WHO_all['Date_registration'])

WHO_covid = pd.read_csv('../data/WHO_covid_from_python.csv')
WHO_covid['Date registration'] = pd.to_datetime(WHO_covid['Date registration'])


WHO_no_covid = pd.read_csv('../data/WHO_no_covid_from_python.csv')
WHO_no_covid['Date_registration'] = WHO_no_covid['Date_registration'].str.replace('"','') 
WHO_no_covid['Date_enrollement'] = WHO_no_covid['Date_enrollement'].str.replace('"','') 
WHO_no_covid['Target_size'] = WHO_no_covid['Target_size'].str.replace('"','')
WHO_no_covid['Date_registration'] = pd.to_datetime(WHO_no_covid['Date_registration'])

In [21]:
#WHO_all.to_excel(r'C:\Users\kkosf\Documents\nss\projects\capstone\data\WHO_from_python.xlsx', engine='xlsxwriter', index=False)

In [14]:
WHO_all.head()

,TrialID,public_title,Scientific_title,study_type,phase,Date_registration,Date_enrollement,Target_size,Recruitment_status,Primary_sponsor,Secondary_sponsors,Countries,Conditions,Interventions,CovidStudy
0,NCT04544813,"""A Study of JNJ-77474462 in Healthy Participants""","""A Phase 1 Study to Investigate the Pharmacoki...","""Interventional""","""Phase 1""",2020-09-09,"October 13, 2020",76,"""Recruiting""","""Janssen Research & Development, LLC""",NaN,"""Australia""","""Healthy""","""Drug: JNJ-77474462;Drug: Anakinra""",False
1,NCT04548999,"""A Study to Evaluate the Efficacy, Safety and ...","""A Phase IIIB Multicenter, Randomized, Double-...","""Interventional""","""Phase 3""",2020-09-09,"December 21, 2020",699,"""Recruiting""","""Hoffmann-La Roche""",NaN,"""United States;Argentina;Belgium;Brazil;Canada...","""Multiple Sclerosis""","""Drug: Ocrelizumab;Drug: Ocrelizumab;Drug: Ant...",False
2,NCT04549051,"""Percutaneous Interruption of the Coracohumera...","""Percutaneous Interruption of the Coracohumera...","""Interventional""","""N/A""",2020-09-09,"November 17, 2020",57,"""Recruiting""","""Albert Einstein College of Medicine""",NaN,"""United States""","""Ashesive Capsulitis""","""Device: Tenex;Drug: Local anesthetic""",False
3,NCT04549337,"""Feasibility and Safety of High-intensity Exer...","""Investigator-blinded Randomized Controlled Tr...","""Interventional""","""N/A""",2020-09-09,"September 25, 2020",10,"""Recruiting""","""Rigshospitalet, Denmark""",NaN,"""Denmark""","""COVID-19""","""Behavioral: Exercise training""",True
4,NCT04550962,"""Description of Characteristics, Such as Age, ...","""PRospEctiVe charactErization of Asthma Patien...","""Observational [Patient Registry]""","""""",2020-09-09,November 2020,800,"""Recruiting""","""Sanofi""","""Regeneron Pharmaceuticals""","""Saudi Arabia;United Arab Emirates;Saudi Arabi...","""Asthma""","""Drug: Dupilumab SAR231893""",False


In [17]:
WHO_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676497 entries, 0 to 676496
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   TrialID             676497 non-null  object        
 1   public_title        676497 non-null  object        
 2   Scientific_title    676497 non-null  object        
 3   study_type          676497 non-null  object        
 4   phase               623907 non-null  object        
 5   Date_registration   675782 non-null  datetime64[ns]
 6   Date_enrollement    672041 non-null  object        
 7   Target_size         654129 non-null  object        
 8   Recruitment_status  676497 non-null  object        
 9   Primary_sponsor     676497 non-null  object        
 10  Secondary_sponsors  145975 non-null  object        
 11  Countries           636335 non-null  object        
 12  Conditions          675573 non-null  object        
 13  Interventions       638635 no

In [6]:
WHO_covid.head()

,TrialID,Public title,Scientific title,Study type,Phase,Date registration,Source Register,Date enrollement,Target size,Recruitment Status,Primary sponsor,Countries,Condition,Intervention
0,NCT04246242,A Randomized Multicenter Controlled Clinical T...,A Randomized Multicenter Controlled Clinical T...,Interventional,Phase 4,2020-01-27,ClinicalTrials.gov,"January 25, 2020",500,Not recruiting,Xiangya Hospital of Central South University,NaN,2019 Novel Coronavirus,Drug: Conventional treatment group;Drug: arbid...
1,NCT04255940,2019-nCoV Outbreak and Cardiovascular Diseases,Impact of a Novel Coronavirus (2019-nCoV) Outb...,Observational,NaN,2020-03-02,ClinicalTrials.gov,"January 20, 2020",12000,Recruiting,Qilu Hospital of Shandong University,China,Cardiovascular Death; Major Adverse Cardiovasc...,NaN
2,NCT04260308,A Survey of Psychological Status of Medical Wo...,A Survey of Psychological Status of Medical Wo...,Observational,NaN,2020-03-02,ClinicalTrials.gov,"February 3, 2020",30000,Recruiting,Huazhong University of Science and Technology,China,Virus; Pneumonia,NaN
3,NCT04260594,Clinical Study of Arbidol Hydrochloride Tablet...,"Randomized, Open, Multicenter Study on the Eff...",Interventional,Phase 4,2020-06-02,ClinicalTrials.gov,"February 7, 2020",380,Not recruiting,Jieming QU,NaN,2019-nCoV,Drug: Arbidol;Other: basic treatment
4,NCT04261426,The Efficacy of Intravenous Immunoglobulin The...,"A Randomized, Open-label, Controlled, Single-c...",Interventional,Phase 2/Phase 3,2020-06-02,ClinicalTrials.gov,"February 10, 2020",80,Not recruiting,Peking Union Medical College Hospital,NaN,2019-nCoV,Drug: Intravenous Immunoglobulin;Other: Standa...


In [7]:
WHO_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7108 entries, 0 to 7107
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   TrialID             7108 non-null   object        
 1   Public title        7108 non-null   object        
 2   Scientific title    7107 non-null   object        
 3   Study type          7108 non-null   object        
 4   Phase               3484 non-null   object        
 5   Date registration   7106 non-null   datetime64[ns]
 6   Source Register     7108 non-null   object        
 7   Date enrollement    7104 non-null   object        
 8   Target size         6994 non-null   object        
 9   Recruitment Status  7084 non-null   object        
 10  Primary sponsor     7096 non-null   object        
 11  Countries           6600 non-null   object        
 12  Condition           6881 non-null   object        
 13  Intervention        6152 non-null   object      

In [8]:
WHO_no_covid.head()

,TrialID,public_title,Scientific_title,study_type,phase,Date_registration,Date_enrollement,Target_size,Recruitment_status,Primary_sponsor,Secondary_sponsors,Countries,Conditions,Interventions,CovidStudy
0,NCT04544813,"""A Study of JNJ-77474462 in Healthy Participants""","""A Phase 1 Study to Investigate the Pharmacoki...","""Interventional""","""Phase 1""",2020-09-09,"October 13, 2020",76,"""Recruiting""","""Janssen Research & Development, LLC""",NaN,"""Australia""","""Healthy""","""Drug: JNJ-77474462;Drug: Anakinra""",False
1,NCT04548999,"""A Study to Evaluate the Efficacy, Safety and ...","""A Phase IIIB Multicenter, Randomized, Double-...","""Interventional""","""Phase 3""",2020-09-09,"December 21, 2020",699,"""Recruiting""","""Hoffmann-La Roche""",NaN,"""United States;Argentina;Belgium;Brazil;Canada...","""Multiple Sclerosis""","""Drug: Ocrelizumab;Drug: Ocrelizumab;Drug: Ant...",False
2,NCT04549051,"""Percutaneous Interruption of the Coracohumera...","""Percutaneous Interruption of the Coracohumera...","""Interventional""","""N/A""",2020-09-09,"November 17, 2020",57,"""Recruiting""","""Albert Einstein College of Medicine""",NaN,"""United States""","""Ashesive Capsulitis""","""Device: Tenex;Drug: Local anesthetic""",False
3,NCT04550962,"""Description of Characteristics, Such as Age, ...","""PRospEctiVe charactErization of Asthma Patien...","""Observational [Patient Registry]""","""""",2020-09-09,November 2020,800,"""Recruiting""","""Sanofi""","""Regeneron Pharmaceuticals""","""Saudi Arabia;United Arab Emirates;Saudi Arabi...","""Asthma""","""Drug: Dupilumab SAR231893""",False
4,NCT04551963,"""Interaction Study of Zanubrutinib With Modera...","""A Drug-Drug Interaction Study of Zanubrutinib...","""Interventional""","""Phase 1""",2020-09-09,"November 20, 2020",30,"""Recruiting""","""BeiGene""",NaN,"""Australia""","""B-cell Malignancies""","""Drug: Zanubrutinib;Drug: Fluconazole;Drug: Di...",False


In [9]:
WHO_no_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669411 entries, 0 to 669410
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   TrialID             669411 non-null  object        
 1   public_title        669411 non-null  object        
 2   Scientific_title    669411 non-null  object        
 3   study_type          669411 non-null  object        
 4   phase               616821 non-null  object        
 5   Date_registration   668698 non-null  datetime64[ns]
 6   Date_enrollement    669411 non-null  object        
 7   Target_size         669411 non-null  object        
 8   Recruitment_status  669411 non-null  object        
 9   Primary_sponsor     669411 non-null  object        
 10  Secondary_sponsors  144326 non-null  object        
 11  Countries           629753 non-null  object        
 12  Conditions          668487 non-null  object        
 13  Interventions       632155 no